In [111]:
import os
from dotenv import load_dotenv
load_dotenv()  
key_string = os.getenv('GEMINI_API_KEY')

In [14]:
import re
from langchain.document_loaders import PyPDFLoader

In [98]:
def pdf_loader(file_path):
    loader = PyPDFLoader(file_path)
    return loader.load()


In [99]:
def pdf_loader(file_path):
    loader = PyPDFLoader(file_path)
    return loader.load()

def page_finder(document):
    start_page = None
    end_page = None

    for doc in document:
        if doc.page_content.startswith("Item 8"):
            start_page = doc.metadata['page']
            break 

    for doc in document:
        if doc.page_content.startswith("Item 9"):
            end_page = doc.metadata['page']
            break 

    if start_page is None or end_page is None:
        raise ValueError("Could not find 'Item 8' or 'Item 9' in the document")

    return start_page, end_page


def page_executor(document):
    start_page, end_page = page_finder(document)
    extracted_documents = [
        doc for doc in document
        if start_page <= doc.metadata['page'] < end_page
    ]

    return extracted_documents

In [101]:
def executor_main():
    pdf_dir = "documents_04_06"
    final_document = []
    pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith('.pdf')]
    for pdf_file in pdf_files:
        document = pdf_loader(os.path.join(pdf_dir, pdf_file))
        extracted_docs = page_executor(document)
        for r in extracted_docs:
            if "CONSOLIDATED STATEMENTS OF OPERATIONS" in r.page_content:
                final_document.append(r.page_content)
    return final_document
    
doc = executor_main()


In [102]:
doc

['CONSOLIDATED STATEMENTS OF OPERATIONS\n(In millions, except number of shares which are reflected in thousands and per share amounts) Years ended  September 24, 2016\xa0 September 26, 2015\xa0 September 27, 2014Net sales$ 215,639\xa0$233,715\xa0$182,795Cost of sales131,376 \xa0140,089\xa0112,258Gross margin84,263 93,626 70,537 \xa0\xa0 \xa0 \xa0\xa0 \xa0Operating expenses:\xa0 \xa0 \xa0\xa0 \xa0Research and development10,045 \xa08,067 \xa06,041 Selling, general and administrative14,194 \xa014,329 \xa011,993 Total operating expenses24,239 22,396 18,034 \xa0\xa0 \xa0 \xa0\xa0 \xa0Operating income60,024 \xa071,230 \xa052,503 Other income/(expense), net1,348 \xa01,285 \xa0980 Income before provision for income taxes61,372 72,515 53,483 Provision for income taxes15,685 \xa019,121 \xa013,973 Net income$ 45,687$53,394$39,510\xa0\xa0 \xa0 \xa0\xa0 \xa0Earnings per share:\xa0 \xa0 \xa0\xa0 \xa0Basic$ 8.35 \xa0$9.28 \xa0$6.49 Diluted$ 8.31 \xa0$9.22 \xa0$6.45 \xa0\xa0 \xa0 \xa0\xa0 \xa0Shares u

In [109]:
from langchain.chains import SequentialChain
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
from langchain.agents import AgentExecutor, Tool
from langchain.agents import ZeroShotAgent

extract_prompt = PromptTemplate(
    input_variables=["document"],
    template="""
You are a financial data extraction assistant.

Your job is to extract the STATEMENTS OF OPERATIONS (also known as Income Statement or Profit & Loss) from the document below into a consistent JSON format, matching standard 10-K structure.
There might be multiple statements of operations in the document.
Document:
{document}

Return a JSON array where each object represents a **single fiscal year** and contains:

[
  {{
    "report_date": "YYYY-MM-DD",
    "section": "profit_and_loss",
    "net_sales": {{
      "products": ...,
      "services": ...,
      "total": ...
    }},
    "cost_of_sales": {{
      "products": ...,
      "services": ...,
      "total": ...
    }},
    "gross_margin": ...,
    "operating_expenses": {{
      "research_and_development": ...,
      "selling_general_and_administrative": ...,
      "total": ...
    }},
    "operating_income": ...,
    "other_income_or_expense_net": ...,
    "income_before_tax": ...,
    "tax_provision": ...,
    "net_income": ...,
    "earnings_per_share": {{
      "basic": ...,
      "diluted": ...
    }},
    "shares_used_in_computing_eps": {{
      "basic": ...,
      "diluted": ...
    }}
  }},
  ...
]

Instructions:
- Parse values from tables and text if needed.
- Normalize field names (e.g., "SG&A" → "selling_general_and_administrative").
- Use `null` for missing values.
- Use `"report_date"` in ISO format: YYYY-MM-DD (e.g., “Sep 30, 2024” → “2024-09-30”).
- Return only the valid JSON array with all numeric values as numbers (not strings).
- please make sure the list of dictionaries returned is in the order of the report_date
- please make sure all the report_dates are covered and we dont miss any report dates
- please make sure we have only one entry for each report date, pritoritize the most complete and recent data for each repeated report date
"""
)


llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",temperature=0, google_api_key=key_string)
# 🧠 Build the single extract chain
extract_chain = LLMChain(
    llm=llm,
    prompt=extract_prompt,
    output_key="json_output"  # customize output key
)

# 🏃 Run the chain

response = extract_chain.run(document=doc)



In [110]:
response = response.strip()
response = response.replace('```json\n', '')
response = response.replace('```', '')
response = response.strip()

import json
structured_data = json.loads(response)

with open('structured_data_all.json', 'w') as f:
    json.dump(structured_data, f, indent=4)
